In [70]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split

import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
from tqdm.notebook import tqdm
import shutil

In [71]:
meta = pd.read_csv('data_origin_paper.csv')
meta

,Unnamed: 0,Image,status
0,5,DDSM Dataset/Malignant Masses\D1_A_1010_1.RIGH...,1
1,11,DDSM Dataset/Malignant Masses\D1_A_1010_1.RIGH...,1
2,17,DDSM Dataset/Malignant Masses\D1_A_1020_1.LEFT...,1
3,23,DDSM Dataset/Malignant Masses\D1_A_1020_1.LEFT...,1
4,29,DDSM Dataset/Malignant Masses\D1_A_1048_1.RIGH...,1
...,...,...,...
2183,13103,DDSM Dataset/Benign Masses\D4_C_0478_1.RIGHT_M...,0
2184,13109,DDSM Dataset/Benign Masses\D4_D_4079_1.RIGHT_C...,0
2185,13115,DDSM Dataset/Benign Masses\D4_D_4079_1.RIGHT_M...,0
2186,13121,DDSM Dataset/Benign Masses\D4_D_4092_1.LEFT_CC...,0


In [72]:
meta.status.value_counts()

status
1    1193
0     995
Name: count, dtype: int64

In [73]:
train_b, test_b = train_test_split(meta[meta.status == 0], test_size=0.5, random_state=42)
train_m, test_m = train_test_split(meta[meta.status == 1], test_size=0.5, random_state=42)

In [74]:
train_m.status.value_counts()

status
1    596
Name: count, dtype: int64

In [75]:
train = pd.concat([train_b, train_m]).sample(frac=1).reset_index()
test = pd.concat([test_b, test_m]).sample(frac=1).reset_index()

In [76]:
train.status.value_counts()

status
1    596
0    497
Name: count, dtype: int64

In [77]:
test.status.value_counts()

status
1    597
0    498
Name: count, dtype: int64

In [78]:
train.to_csv('train.csv')

In [79]:
test.to_csv('test.csv')

In [80]:
len(train)

1093

In [81]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [82]:
len(train)

1093

In [83]:
len(test)

1095

In [89]:
data = []
# Image roatation_120  roatation_240  clahe
for i in train.index:
    # Get image origin
    data.append([train.Image[i], 0, 0, 0, train.status[i]])
    data.append([train.Image[i], 0, 0, 1, train.status[i]])
    # Get image roatation 120
    data.append([train.Image[i], 1, 0, 0, train.status[i]])
    data.append([train.Image[i], 1, 0, 1, train.status[i]])
    # Get image roatation 240
    data.append([train.Image[i], 0, 1, 0, train.status[i]])
    data.append([train.Image[i], 0, 1, 1, train.status[i]])

In [90]:
meta_new = pd.DataFrame(columns= ['Image', 'roatation_120',  'roatation_240' , 'clahe', 'status'], data=data)

In [91]:
meta_new

,Image,roatation_120,roatation_240,clahe,status
0,DDSM Dataset/Benign Masses\D3_D_4082_1.LEFT_ML...,0,0,0,0
1,DDSM Dataset/Benign Masses\D3_D_4082_1.LEFT_ML...,0,0,1,0
2,DDSM Dataset/Benign Masses\D3_D_4082_1.LEFT_ML...,1,0,0,0
3,DDSM Dataset/Benign Masses\D3_D_4082_1.LEFT_ML...,1,0,1,0
4,DDSM Dataset/Benign Masses\D3_D_4082_1.LEFT_ML...,0,1,0,0
...,...,...,...,...,...
6553,DDSM Dataset/Benign Masses\D3_B_3357_1.LEFT_ML...,0,0,1,0
6554,DDSM Dataset/Benign Masses\D3_B_3357_1.LEFT_ML...,1,0,0,0
6555,DDSM Dataset/Benign Masses\D3_B_3357_1.LEFT_ML...,1,0,1,0
6556,DDSM Dataset/Benign Masses\D3_B_3357_1.LEFT_ML...,0,1,0,0


In [88]:
meta_new.to_csv('train_aug.csv')